一些全局数据,lim表示follower数量的范围

In [214]:
lim=4000
data_path='./data/'

先建立用户名到index的映射

In [215]:
import os
import csv

user_ind={}
user=[]
with open(os.path.join(data_path,f'followers_over_{lim}.csv'),mode='r',encoding='utf-8') as f:
    user=list(map(lambda x:x[0],csv.reader(f)))
    for i,name in enumerate(user):
        user_ind[name]=i
    csv_reader=csv.reader(f)
print(user_ind)
print(user)
print(len(user))

{'torvalds': 0, 'yyx990803': 1, 'gaearon': 2, 'ruanyf': 3, 'gustavoguanabara': 4, 'peng-zhihui': 5, 'openai': 6, 'bradtraversy': 7, 'JakeWharton': 8, 'sindresorhus': 9, 'karpathy': 10, 'microsoft': 11, 'tj': 12, 'rafaballerini': 13, 'getify': 14, 'addyosmani': 15, 'geohot': 16, 'llSourcell': 17, 'michaelliao': 18, 'google': 19, 'github': 20, 'wesbos': 21, 'kentcdodds': 22, 'paulirish': 23, 'filipedeschamps': 24, 'ry': 25, 'kamranahmedse': 26, 'taylorotwell': 27, 'TheAlgorithms': 28, 'diego3g': 29, 'iamshaunjp': 30, 'kennethreitz': 31, 'benawad': 32, 'StephenGrider': 33, 'WebDevSimplified': 34, 'buckyroberts': 35, '3b1b': 36, 'antfu': 37, 'kunal-kushwaha': 38, 'dmalan': 39, 'hadley': 40, 'huggingface': 41, 'daimajia': 42, 'EpicGames': 43, 'mojombo': 44, 'sdras': 45, 'mdo': 46, 'krishnaik06': 47, 'mbostock': 48, 'mrdoob': 49, 'lucidrains': 50, 'defunkt': 51, 'ThePrimeagen': 52, 'tiangolo': 53, 'douglascrockford': 54, 'kelseyhightower': 55, 'gvanrossum': 56, 'gamemann': 57, 'jwasham': 58,

基于之前获得的用户的关注列表，star仓库列表和拥有的仓库列表，计算一个用户对于另外一个用户的权重，这里对于权重进行说明：

考虑$A->B$的权重：
* $following$ : A一共following的用户的数量   
* $star$ : A一共star的仓库的数量
* $k_{1}$ : A,B共同star的仓库的权重     
* $num_{1}$ : A,B共同star的仓库的数量
* $k_{2}$ : A star B的仓库的权重      
* $num_{2}$ ：A star B的仓库的数量
* $k_{3}$ : A,B共同following的用户的权重   
* $num_{3}$ ：A,B共同following的用户数量
* $k_{4}$ : A关注B的权重
* $k_{5}$ ：AB相互关注的权重

所以最后的权重的计算公式为：
$weight_{A->B}=k_{1}*num_{1}/star+k_{2}*num_{2}/star+k_{3}*num_{3}/following+k_{4}+k_{5}$

在这里，我对于$following,star$的数量进行了一个限制，因为我认为star或者following的人数较少的话，可能不能够有效地体现出该用户的偏好。

对于权重的设置，这里经过了一些尝试，我认为目前的设置应该是最好的，简单介绍一下思路: 我认为A star B的仓库的权重，和AB相互关注的权重应该是最高的，因为这边表明两人的关系非常紧密，共同star的较低，因为有可能存在从众效应。A关注B的权重为0，因为我认为A关注B不能说明很多问题，比方说Linus的仓库有很多人关注，但是从事Linux相关工作的用户其实非常少。

最后设置一个门槛，表明只有关系达到一定的程度，才被认为两者之间存在关系

In [216]:
from tqdm import tqdm

total = len(user)

k1 = 100  # 共同star的仓库 的权重
k2 = 2000  # A star B的仓库 的权重
k3 = 100  # 共同following的ueser 的权重
k4 = 0 # A关注B 的权重
k5 = 500  # AB相互关注 的权重
threshhold = 50

st = set()
cnt=0

def fun(reader):
    return list(map(lambda x: x[0], reader))

miss = set()

following_list = []
for i in tqdm(range(0, total)):
    if not os.path.exists(os.path.join(data_path, f"{user[i]}_following.csv")):
        following_list.append({})
        miss.add(i)
    else:
        with open(
            os.path.join(data_path, f"{user[i]}_following.csv"),
            mode="r",
            encoding="utf-8",
        ) as f:
            following_list.append(set(fun(csv.reader(f))))

star_list = []
for i in tqdm(range(0, total)):
    if not os.path.exists(os.path.join(data_path, f"{user[i]}_stars.csv")):
        star_list.append({})
        miss.add(i)
    else:
        with open(
            os.path.join(data_path, f"{user[i]}_stars.csv"), mode="r", encoding="utf-8"
        ) as f:
            star_list.append(set(fun(csv.reader(f))))

repo_list = []
for i in tqdm(range(0, total)):
    if not os.path.exists(os.path.join(data_path, f"{user[i]}_repos.csv")):
        repo_list.append({})
        miss.add(i)
    else:
        with open(
            os.path.join(data_path, f"{user[i]}_repos.csv"), mode="r", encoding="utf-8"
        ) as f:
            repo_list.append(set(fun(csv.reader(f))))

with open("./graph.txt", mode="w", encoding="utf-8") as f:
    for i in tqdm(range(0, total)):
        if i in miss:
            continue

        for j in range(0, total):
            if i == j or j in miss:
                continue

            weight = 0
            # 共同star的仓库
            if len(star_list[i]) > 50:
                weight += (
                    k1
                    * len(star_list[i].intersection(star_list[j]))
                    / len(star_list[i])
                )

            # A star B的仓库
            if len(star_list[i]) > 50:
                weight += (
                    k2
                    * len(star_list[i].intersection(repo_list[j]))
                    / len(star_list[i])
                )
            # 共同following的user
            if len(following_list[i]) > 30:
                weight += (
                    k3
                    * len(following_list[i].intersection(following_list[j]))
                    / len(following_list[i])
                )
            # A关注B
            weight += k4 if user[j] in following_list[i] else 0
            # AB相互关注
            weight += (
                k5
                if user[j] in following_list[i] and user[i] in following_list[j]
                else 0
            )

            if user[j] in following_list[i] and user[i] in following_list[j] and i<j and user[i]=='moxie0':
                print(user[i],user[j])

            if weight >= threshhold:
                f.write(f"{user[i]} {user[j]} {int(weight)}\n")
                cnt+=1
                st.add(user[i])
                st.add(user[j])

print(len(st),cnt)

  0%|          | 0/953 [00:00<?, ?it/s]

100%|██████████| 953/953 [00:06<00:00, 150.52it/s]

661 7080


利用pickle，将有关的数据保存，便于之后使用

In [217]:
import pickle
pickle.dump([user,user_ind,following_list,star_list,repo_list],open('./data.pkl','wb'))